### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [ ]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.get_statistics import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.superpixel import *
from Pipeline.pre_processing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import preprocessing_config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

### Loading the data

In [ ]:
# temp_pipeline = PreProcessingPipelineTemporal()

In [ ]:
# train_fn, dataloader_train = temp_pipeline.get_processed_trainloader(64, 'indexbands', vi_type='msi')

Train

In [ ]:
temporal_images = load_field_images_temporal(config.base_directory_temporal_train)
border_removed_images_train = blacken_field_borders_temporal(temporal_images)
field_numbers_train, indices_images_train = allbands_temporal_cubes(border_removed_images_train)

len(indices_images_train), indices_images_train[0][0].shape

(2425, (64, 64, 10))

Test

In [ ]:
temporal_images_test = load_field_images_temporal(config.base_directory_temporal_test)
border_removed_images = blacken_field_borders_temporal(temporal_images_test)
field_numbers_test, indices_images_test = allbands_temporal_cubes(border_removed_images)

(len(indices_images_test), indices_images_test[0][0].shape)

(48, (64, 64, 10))

In [ ]:
image_tensor_train = np.stack(indices_images_train)  # Shape: (N x 7 x 64 x 64 x 6)
image_tensor_test = np.stack(indices_images_test)   # Shape: (N x 7 x 64 x 64 x 6)

image_tensor_train.shape, image_tensor_test.shape

((2425, 7, 64, 64, 10), (48, 7, 64, 64, 10))

### Super-patches

In [ ]:
image_tensor_train = torch.tensor(image_tensor_train, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_test = torch.tensor(image_tensor_test, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_train.shape, image_tensor_test.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

In [ ]:
train_patches, train_patch_coordinates = non_overlapping_sliding_window(image_tensor_train, field_numbers_train, patch_size=4)
test_patches, test_patch_coordinates = non_overlapping_sliding_window(image_tensor_test, field_numbers_test, patch_size=4)
len(test_patches), test_patches[0].shape

### Single Timestamp Images